In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cfj7bs1w
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cfj7bs1w
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-no73q7sx/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
!nvidia-smi

Mon Oct 21 18:55:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!wget "https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/CryptForCuda.c"

--2024-10-21 18:55:30--  https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/CryptForCuda.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1642 (1.6K) [text/plain]
Saving to: ‘CryptForCuda.c’

CryptForCuda.c      100%[===================>]   1.60K  --.-KB/s    in 0s      

2024-10-21 18:55:30 (35.2 MB/s) - ‘CryptForCuda.c’ saved [1642/1642]



Encryption happens below

In [5]:
# Save the code into a .cu file
%%writefile encrypt.cu
#include <stdio.h>
#include <stdlib.h>

__device__ void copyString(char* dest, const char* src, int length) {
    for (int i = 0; i < length; i++) {
        dest[i] = src[i];
    }
    dest[length] = '\0'; // Null terminate the string
}

__device__ char* cudaCrypt(const char* rawPassword) {
    static char newPassword[11]; // Use static as a local pointer should not be returned

    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    for (int i = 0; i < 10; i++) {
        if (i >= 0 && i < 6) { // Checking all lower case letter limits
            if (newPassword[i] > 122) {
                newPassword[i] = (newPassword[i] - 122) + 97;
            } else if (newPassword[i] < 97) {
                newPassword[i] = (97 - newPassword[i]) + 97;
            }
        } else { // Checking number section
            if (newPassword[i] > 57) {
                newPassword[i] = (newPassword[i] - 57) + 48;
            } else if (newPassword[i] < 48) {
                newPassword[i] = (48 - newPassword[i]) + 48;
            }
        }
    }

    return newPassword;
}

__global__ void encryptPasswords(const char* rawPasswords, char* encryptedPasswords, int numPasswords) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < numPasswords) {
        char localRaw[5]; // To hold the password including null terminator
        for (int i = 0; i < 4; i++) {
            localRaw[i] = rawPasswords[idx * 4 + i];
        }
        localRaw[4] = '\0'; // Null terminate the string

        char* encryptedPass = cudaCrypt(localRaw);

        // Copy the encrypted password to the output array
        for (int j = 0; j < 11; j++) {
            encryptedPasswords[idx * 11 + j] = encryptedPass[j];
        }
    }
}

int main() {
    const int numPasswords = 1; // Number of passwords to encrypt
    const int passwordLength = 4; // Length of each password
    const int encryptedLength = 11; // Length of each encrypted password

    // Allocate host memory for passwords with enough space for null terminator
    char* h_rawPasswords = (char*)malloc(numPasswords * (passwordLength + 1) * sizeof(char)); // Allocating size for "hp93" + '\0'
    char* h_encryptedPasswords = (char*)malloc(numPasswords * encryptedLength * sizeof(char));

    // Initialize the input password
    strcpy(h_rawPasswords, "hp93");

    // Allocate device memory
    char* d_rawPasswords;
    char* d_encryptedPasswords;

    cudaMalloc((void**)&d_rawPasswords, numPasswords * (passwordLength + 1) * sizeof(char));
    cudaMalloc((void**)&d_encryptedPasswords, numPasswords * encryptedLength * sizeof(char));

    // Copy data from host to device
    cudaMemcpy(d_rawPasswords, h_rawPasswords, numPasswords * (passwordLength + 1) * sizeof(char), cudaMemcpyHostToDevice);

    // Launch kernel with one block and one thread per password
    encryptPasswords<<<1, numPasswords>>>(d_rawPasswords, d_encryptedPasswords, numPasswords);

    // Copy encrypted passwords back to host
    cudaMemcpy(h_encryptedPasswords, d_encryptedPasswords, numPasswords * encryptedLength * sizeof(char), cudaMemcpyDeviceToHost);

    // Print the encrypted password
    printf("Encrypted Password: %s\n", h_encryptedPasswords);

    // Free device memory
    cudaFree(d_rawPasswords);
    cudaFree(d_encryptedPasswords);

    // Free host memory
    free(h_rawPasswords);
    free(h_encryptedPasswords);

    return 0;
}

Writing encrypt.cu


In [6]:
!nvcc encrypt.cu -o encrypt
!./encrypt

Encrypted Password: jfismo2771


The Encrypted password value is seen on above

In [7]:
# Save the code into a .cu file
%%writefile password_crack.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX_PASSWORD_LEN 11
#define BLOCK_SIZE 256
#define GRID_SIZE 256

__device__ char* CudaCrypt(char* rawPassword) {
    char* newPassword = (char*)malloc(sizeof(char) * 11);

    newPassword[0] = rawPassword[0] + 2;
    newPassword[1] = rawPassword[0] - 2;
    newPassword[2] = rawPassword[0] + 1;
    newPassword[3] = rawPassword[1] + 3;
    newPassword[4] = rawPassword[1] - 3;
    newPassword[5] = rawPassword[1] - 1;
    newPassword[6] = rawPassword[2] + 2;
    newPassword[7] = rawPassword[2] - 2;
    newPassword[8] = rawPassword[3] + 4;
    newPassword[9] = rawPassword[3] - 4;
    newPassword[10] = '\0';

    for(int i = 0; i < 10; i++) {
        if(i >= 0 && i < 6) {
            if(newPassword[i] > 122) {
                newPassword[i] = (newPassword[i] - 122) + 97;
            } else if(newPassword[i] < 97) {
                newPassword[i] = (97 - newPassword[i]) + 97;
            }
        } else {
            if(newPassword[i] > 57) {
                newPassword[i] = (newPassword[i] - 57) + 48;
            } else if(newPassword[i] < 48) {
                newPassword[i] = (48 - newPassword[i]) + 48;
            }
        }
    }
    return newPassword;
}

__device__ int comparePasswords(char* generated, char* target) {
    for (int i = 0; i < MAX_PASSWORD_LEN - 1; i++) {
        if (generated[i] != target[i]) {
            return 0;
        }
    }
    return 1;
}

__global__ void crack(char* target, char* result, int* found) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;
    int idz = blockIdx.z * blockDim.z + threadIdx.z;

    if (*found) return;

    char attempt[5];
    attempt[0] = 'a' + idx % 26;
    attempt[1] = 'a' + idy % 26;
    attempt[2] = '0' + idz / 10;
    attempt[3] = '0' + idz % 10;
    attempt[4] = '\0';

    char* encrypted = CudaCrypt(attempt);

    if (comparePasswords(encrypted, target)) {
        *found = 1;
        for (int i = 0; i < 4; i++) {
            result[i] = attempt[i];
        }
        result[4] = '\0';
    }

    free(encrypted);
}

int main(int argc, char** argv) {
    if (argc != 2) {
        printf("Usage: %s <encrypted_password>\n", argv[0]);
        return 1;
    }

    char* h_target = argv[1];
    char* d_target;
    char* d_result;
    int* d_found;
    char h_result[5];
    int h_found = 0;

    cudaMalloc((void**)&d_target, MAX_PASSWORD_LEN * sizeof(char));
    cudaMalloc((void**)&d_result, 5 * sizeof(char));
    cudaMalloc((void**)&d_found, sizeof(int));

    cudaMemcpy(d_target, h_target, MAX_PASSWORD_LEN * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_found, &h_found, sizeof(int), cudaMemcpyHostToDevice);

    dim3 block(8, 8, 4);
    dim3 grid(32, 32, 25);

    crack<<<grid, block>>>(d_target, d_result, d_found);

    cudaDeviceSynchronize();

    cudaMemcpy(&h_found, d_found, sizeof(int), cudaMemcpyDeviceToHost);
    if (h_found) {
        cudaMemcpy(h_result, d_result, 5 * sizeof(char), cudaMemcpyDeviceToHost);
        printf("Password found: %s\n", h_result);
    } else {
        printf("Password not found\n");
    }

    cudaFree(d_target);
    cudaFree(d_result);
    cudaFree(d_found);

    return 0;
}

Writing password_crack.cu


Then here i execute the decrypt code above by give the above generated encrypted password

In [8]:
!nvcc -o password_crack password_crack.cu
!./password_crack "jfismo2771"


Password found: hp93
